### Import Required Package

In [70]:
import pandas as pd
import talib
import numpy as np
import json

### Import Tidal

In [71]:
import tidal as td

### Initialize Plumber

In [72]:
from pathlib import Path
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

PLUMBER_HOST = "https://dev-api.ddt-dst.cc/api/plumber/"
with open(f'{str(Path.home())}/.config/gcloud/application_default_credentials.json') as plumber_token:
    token = json.load(plumber_token)

### Load market data and compute technical indicators

In [73]:
import os
import pandas as pd

os.chdir('/home/jovyan/individualized-indicator')
# 定義你想要讀取檔案的目錄
directory = './emb'

start_date = "2022-06-01"
end_date = "2024-09-30"

# 初始化一個空的字典來存儲所有讀取的 DataFrame
dataframes = {}

# 遍歷目錄中的所有檔案
for filename in os.listdir(directory):
    if filename.endswith('.pkl'):
        # 組合完整的檔案路徑
        file_path = os.path.join(directory, filename)
        # 讀取 .pkl 檔案
        df = pd.read_pickle(file_path)
        
        # 使用檔案名稱（不包括副檔名）作為字典的鍵
        variable_name = filename.split('_')[2]
        
        # 將 DataFrame 儲存到字典中
        dataframes[variable_name] = df

keys_list = list(dataframes.keys())
# 將字符串轉換為 datetime 格式，然後進行排序
sorted_dates = sorted(pd.to_datetime(keys_list))
keys_list = [date.strftime('%Y-%m-%d') for date in sorted_dates]

total_df = pd.DataFrame()

for j in range(0,len(keys_list)):
    df_name = keys_list[j]
    selected_df = dataframes[df_name]    
    selected_df = selected_df.reset_index()
    stock_list = pd.DataFrame(selected_df.iloc[:,0].unique())
    total_df = pd.concat([total_df, stock_list], ignore_index=True)
    
unique_values_list = list(total_df.iloc[:,0].unique())
sorted_list = sorted(unique_values_list)

total_data = pd.read_parquet(
    f"{PLUMBER_HOST}stocks/tw/ohlcv",
    storage_options={
        "gcp-token": json.dumps(token),
        "start-date": start_date,
        "end-date": end_date,
        "tickers": ",".join([stock for stock in sorted_list]),
    },
)

total_data.index = total_data.index.set_levels(
    pd.to_datetime(total_data.index.levels[1]),level=1)

total_data.rename_axis(index={
    'ticker': 'instrument'
}, inplace=True)

total_data['new'] = -1
total_data.reset_index(inplace=True)

def filter_by_year_month_12(filter_data, select_year, select_mon):
    # 計算當前篩選年份和下一個年份
    next_year = select_year + 1
    
    # 構建篩選條件：當前年12月、次年1月、次年2月
    filter_data = filter_data[
        ((filter_data['datetime'].dt.year == next_year) & (filter_data['datetime'].dt.month == 1)) |
        ((filter_data['datetime'].dt.year == next_year) & (filter_data['datetime'].dt.month == 2)) |
        ((filter_data['datetime'].dt.year == next_year) & (filter_data['datetime'].dt.month == 3))
    ]
    
    return filter_data

concat_df = pd.DataFrame()

# for i in range(0,len(keys_list)-1):
for i in range(0,len(keys_list)):
    df_name = keys_list[i]
    selected_df = dataframes[df_name]
    selected_df.reset_index(inplace=True)
    stock_df = selected_df.iloc[:,0]
    stock_df_unique = list(stock_df.unique())
    result = list(set(sorted_list) - set(stock_df_unique))
    filtered_df = total_data[total_data['instrument'].isin(result)]
    filtered_df = filtered_df.rename(columns={'volume': 'vol'})
    selected_df = pd.concat([selected_df, filtered_df], ignore_index=True)
    
    select_year = int(df_name.split('-')[0])
    select_mon = int(df_name.split('-')[1])
    select_mons = [(select_mon+1)%12, (select_mon+2)%12, (select_mon+3)%12]
    select_mons = [12 if mon == 0 else mon for mon in select_mons]
    
    selected_df['datetime'] = pd.to_datetime(selected_df['datetime'])
    # 過濾出所有6月的交易資料
    filter_data = selected_df[selected_df['datetime'].dt.month.isin(select_mons)]

    # 使用時，根據 select_year 和 select_mon 動態篩選數據
        
    if select_mon == 12:
        filter_data = filter_by_year_month_12(filter_data, select_year, select_mon) 

    else:
        filter_data = filter_data[filter_data['datetime'].dt.year == select_year]
    
    concat_df = pd.concat([concat_df, filter_data], ignore_index=True)
    
# i = len(keys_list)-1
# df_name = keys_list[i]
# selected_df = dataframes[df_name]
# selected_df.reset_index(inplace=True)
# stock_df = selected_df.iloc[:,0]
# stock_df_unique = list(stock_df.unique())
# result = list(set(sorted_list) - set(stock_df_unique))
# filtered_df = total_data[total_data['instrument'].isin(result)]
# filtered_df = filtered_df.rename(columns={'volume': 'vol'})
# selected_df = pd.concat([selected_df, filtered_df], ignore_index=True)
# concat_df = pd.concat([concat_df, selected_df], ignore_index=True)

concat_df.sort_values(by=['instrument', 'datetime'], inplace=True)
concat_df.reset_index(drop=True, inplace=True)
row = np.where(concat_df.iloc[:,0] == '6251')[0]
concat_df = concat_df.drop(index=row)
concat_df = concat_df.reset_index(drop=True)
row = np.where(concat_df.iloc[:,0] == '8406')[0]
concat_df = concat_df.drop(index=row)
concat_df = concat_df.reset_index(drop=True)
row = np.where(concat_df.iloc[:,0] == '6548')[0]
concat_df = concat_df.drop(index=row)
concat_df = concat_df.reset_index(drop=True)
quote_data = concat_df.set_index(['instrument', 'datetime'])
quote_data.columns.values[4] = 'volume'

# quote_data.to_csv('./quote_data.csv')

/tmp/ipykernel_56642/4042907721.py:97: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior. In a future version these will be considered non-comparable. Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  selected_df['datetime'] = pd.to_datetime(selected_df['datetime'])
/tmp/ipykernel_56642/4042907721.py:97: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior. In a future version these will be considered non-comparable. Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  selected_df['datetime'] = pd.to_datetime(selected_df['datetime'])
/tmp/ipykernel_56642/4042907721.py:97: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior. In a future version these will be considered non-comparable. Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  selected_d

In [74]:
# i = len(keys_list)-1
# df_name = keys_list[i]
# selected_df = dataframes[df_name]
# selected_df.reset_index(inplace=True)
# stock_df = selected_df.iloc[:,0]
# stock_df_unique = list(stock_df.unique())
# result = list(set(sorted_list) - set(stock_df_unique))
# filtered_df = total_data[total_data['instrument'].isin(result)]
# filtered_df = filtered_df.rename(columns={'volume': 'vol'})
# selected_df = pd.concat([selected_df, filtered_df], ignore_index=True)
# concat_df = pd.concat([concat_df, selected_df], ignore_index=True)

### Strategy Lake API

In [75]:
data_api_url = ("https://dev-api.ddt-dst.cc/api/plumber/nextron/industry_prod_df")
df = pd.read_parquet(
    data_api_url,
    storage_options={
        "gcp-token": json.dumps(token),
    }
)

### Load benchmark data

In [76]:
benchmark_inst = "Y9999"
benchmark_data = pd.read_parquet(
    f"{PLUMBER_HOST}stocks/tw/ohlcv",
    storage_options={
        "gcp-token": json.dumps(token),
        "start-date": start_date,
        "end-date": end_date,
        "tickers": benchmark_inst,
    },
)
benchmark_data.index.set_levels(
    pd.to_datetime(benchmark_data.index.levels[1]),
    level=1,
    inplace=True,
)
benchmark_data.rename_axis(index={
    'ticker': 'instrument'
}, inplace=True)

benchmark_data

/tmp/ipykernel_56642/1318227356.py:11: FutureWarning: inplace is deprecated and will be removed in a future version.
  benchmark_data.index.set_levels(


open      high       low     close      volume
instrument datetime                                                      
Y9999      2022-06-01  16718.91  16811.04  16649.91  16675.09  5693916000
           2022-06-02  16605.46  16617.26  16540.55  16552.57  4836099000
           2022-06-06  16570.89  16654.12  16509.01  16605.96  5418061000
           2022-06-07  16593.75  16593.75  16465.87  16512.88  4732466000
           2022-06-08  16538.67  16702.99  16538.67  16670.51  4815479000
...                         ...       ...       ...       ...         ...
           2024-09-24  22273.79  22431.78  22154.55  22431.78  7862076000
           2024-09-25  22569.29  22785.32  22569.29  22761.60  9143783000
           2024-09-26  22841.42  22961.77  22769.24  22858.81  9160464000
           2024-09-27  22978.97  23079.21  22806.83  22822.79  9742471000
           2024-09-30  22643.91  22691.23  22224.54  22224.54  9221061000

[569 rows x 5 columns]

### Initialize Tidal
1. Initialize Tidal object
2. Add Quote data (pd.DataFrame)
3. Set strategy object (td.BaseStrategy)
4. Add metric objects (td.BaseMetic)

In [77]:
# Initialize Tidal object
tidal = td.Tidal(init_cash=50000000, slip_ticks=1, stock_config=td.StockConfig.TW, load_configs=True, reqMem="1000Mi", ignore_volume_size=True)

# Add quote data
tidal.add_quote(quote_data)

# Set strategy object
tidal.set_strategy(td.strategy.TopkDropout(15, 2, 'new'))

# Set metric objects
tidal.add_metric(td.metric.AccountInfo())
tidal.add_metric(td.metric.AdditionalInfo())
tidal.add_metric(td.metric.PositionInfo())
tidal.add_metric(td.metric.Portfolio(benchmark_data.loc[benchmark_inst]))

### Stock Config

In [78]:
tidal.exchange.stock_config

InstConfig {Margin:0.0, Tick Size:0.002, Tick Value:0.002, Trade Unit:1000, Commission:0.0004275, Min Commission:20.0, Transaction Tax:0.003}

### Instrument Configs

In [79]:
tidal.exchange.inst_configs

{'MTX': InstConfig {Margin:46000.0, Tick Size:1.0, Tick Value:50.0, Trade Unit:1, Commission:20.0, Min Commission:0.0, Transaction Tax:2e-05},
 'TX': InstConfig {Margin:184000.0, Tick Size:1.0, Tick Value:200.0, Trade Unit:1, Commission:40.0, Min Commission:0.0, Transaction Tax:2e-05},
 'NQ': InstConfig {Margin:16500.0, Tick Size:0.25, Tick Value:5.0, Trade Unit:1, Commission:1.85, Min Commission:0.0, Transaction Tax:0.0}}

### Config Modification

In [80]:
# Set commission to 77% off
tidal.exchange.set_stock_config(commission=0.001425 * 0.23)
tidal.exchange.stock_config

InstConfig {Margin:0.0, Tick Size:0.002, Tick Value:0.002, Trade Unit:1000, Commission:0.00032775, Min Commission:20.0, Transaction Tax:0.003}

### Add New Instrument Config

In [81]:
# Add MGC config
tidal.exchange.set_config(instrument='MGC', margin=787., tick_size=0.1, tick_value=1., trade_unit=1, commission=2., min_commission=0., transaction_tax=0.)
tidal.exchange.inst_configs

{'MTX': InstConfig {Margin:46000.0, Tick Size:1.0, Tick Value:50.0, Trade Unit:1, Commission:20.0, Min Commission:0.0, Transaction Tax:2e-05},
 'TX': InstConfig {Margin:184000.0, Tick Size:1.0, Tick Value:200.0, Trade Unit:1, Commission:40.0, Min Commission:0.0, Transaction Tax:2e-05},
 'NQ': InstConfig {Margin:16500.0, Tick Size:0.25, Tick Value:5.0, Trade Unit:1, Commission:1.85, Min Commission:0.0, Transaction Tax:0.0},
 'MGC': InstConfig {Margin:787.0, Tick Size:0.1, Tick Value:1.0, Trade Unit:1, Commission:2.0, Min Commission:0.0, Transaction Tax:0.0}}

### Start Backtesting

In [82]:
tidal.backtest()
logs = tidal.get_logs()
for log in logs:
    print(log)

Tidal Backtesting: 100%|██████████| 548/548 [01:06<00:00,  8.30it/s, cash=4.84e+5, pnl=6.19e+5, position_cost=6.34e+7, value=6.45e+7] 

2023-10-17 - 警告：強制以開盤價 26.8733 賣出 5220，因為當天交易量為0


### Metric - AccountInfo

In [83]:
account_info = tidal.metrics["AccountInfo"].report
print(account_info)

                    cash    trade_cost  position_cost     slip_cost  \
datetime                                                              
2022-07-01  5.000000e+07  0.000000e+00   0.000000e+00  0.000000e+00   
2022-07-04  5.000000e+07  0.000000e+00   0.000000e+00  0.000000e+00   
2022-07-05  2.342548e+06  1.561461e+04   4.764184e+07  5.906313e+05   
2022-07-06  2.342548e+06  1.561461e+04   4.764184e+07  5.906313e+05   
2022-07-07  2.498338e+05  2.757563e+04   4.963455e+07  6.334381e+05   
...                  ...           ...            ...           ...   
2024-09-24  4.959362e+05  1.047721e+07   6.356097e+07  1.204556e+07   
2024-09-25  5.595247e+05  1.050759e+07   6.365230e+07  1.206648e+07   
2024-09-26  3.760204e+05  1.053648e+07   6.363830e+07  1.211084e+07   
2024-09-27  4.839932e+05  1.056619e+07   6.341688e+07  1.210665e+07   
2024-09-30  4.839932e+05  1.056619e+07   6.341688e+07  1.210665e+07   

                     pnl         value  max_drawdown      turnover  
datetim

### Metric - PositionInfo

In [84]:
position_df = tidal.metrics["PositionInfo"].report
print(position_df)

                       quantity       price   commission   slip_cost  \
instrument datetime                                                    
1102       2023-10-12   98000.0   38.557561  1238.449587  12216.7976   
1308       2023-07-19  155000.0   24.775953  1258.649382  29365.2150   
           2023-07-20  155000.0   24.775953  1258.649382  29365.2150   
           2023-07-28  206000.0   25.210700  1702.138227  10073.4000   
           2023-07-31  206000.0   25.210700  1702.138227  10073.4000   
...                         ...         ...          ...         ...   
9942       2024-04-30   32000.0  119.944400  1257.976867  30224.0000   
           2024-05-02   32000.0  119.944400  1257.976867  30224.0000   
           2024-05-03   32000.0  119.944400  1257.976867  30224.0000   
           2024-05-06   32000.0  119.944400  1257.976867  30224.0000   
           2024-05-07   32000.0  119.944400  1257.976867  30224.0000   

                               pnl  now_price  
instrument date

In [85]:
pi_report = tidal.metrics['PositionInfo'].report
pi_report.iloc[pi_report.index.get_level_values('datetime') == '2022-06-14']

,,quantity,price,commission,slip_cost,pnl,now_price
instrument,datetime,,,,,,


### Metric - AdditionalInfo

In [86]:
tidal.metrics['AdditionalInfo'].report

new
instrument datetime            
1101       2022-07-01  0.000566
           2022-07-04  0.000193
           2022-07-05  0.000557
           2022-07-06  0.000054
           2022-07-07  0.000682
...                         ...
9958       2024-09-24  0.001409
           2024-09-25  0.000066
           2024-09-26  0.000557
           2024-09-27  0.000468
           2024-09-30  0.000280

[332693 rows x 1 columns]

### Strategy Lake Submit

In [87]:
# group = dict(zip(df["coid"].astype(str), df["tejind4_c"]))
# submit_lake_backtest_result = tidal.submit_lake_backtest(account_info=account_info, position_df=position_df, benchmark_info=benchmark_data, group=group,lake_env="dev",strategy_id=19)
# print(submit_lake_backtest_result)

### Trade Report

In [88]:
tidal.trade_report

,win_num,lose_num,trade_num,pos_num,win_rate,profit,loss,trade_cost,pnl
instrument,,,,,,,,,
6139,5.0,2.0,7.0,0.0,0.714286,5.295425e+06,-2.655211e+05,1.147235e+05,5.029903e+06
2392,2.0,0.0,2.0,0.0,1.000000,2.453756e+06,0.000000e+00,3.817314e+04,2.453756e+06
3617,4.0,5.0,9.0,0.0,0.444444,2.386635e+06,-4.475813e+05,1.244981e+05,1.939054e+06
3551,11.0,8.0,19.0,0.0,0.578947,2.495493e+06,-5.939452e+05,2.664059e+05,1.901548e+06
4129,3.0,1.0,4.0,0.0,0.750000,1.741897e+06,-8.098819e+02,5.427837e+04,1.741087e+06
...,...,...,...,...,...,...,...,...,...
3038,0.0,10.0,10.0,0.0,0.000000,0.000000e+00,-1.400797e+06,1.400088e+05,-1.400797e+06
4961,5.0,15.0,20.0,0.0,0.250000,1.044265e+06,-2.583664e+06,2.611685e+05,-1.539399e+06
6829,3.0,12.0,15.0,0.0,0.200000,6.686880e+05,-2.400360e+06,2.048908e+05,-1.731672e+06


### Traded instruments

In [89]:
tidal.account.trades.keys()

dict_keys(['3669', '2640', '3617', '8433', '6245', '1817', '4549', '5230', '1531', '1580', '3551', '8928', '4129', '2441', '2207', '4971', '4961', '6612', '2030', '6533', '2850', '6284', '8478', '6829', '6568', '8027', '5439', '8091', '3567', '1533', '1565', '8109', '3556', '6667', '5220', '3141', '3006', '4945', '8942', '4576', '6747', '3479', '6591', '4506', '3611', '8279', '5903', '4175', '4541', '4162', '6803', '4938', '2455', '3218', '6589', '5284', '2404', '6470', '6664', '6788', '4927', '6670', '3413', '6143', '1589', '4931', '3484', '3264', '8049', '1504', '3189', '6416', '6269', '4721', '2421', '1712', '5443', '2337', '8420', '6570', '8210', '3038', '6139', '3029', '4953', '1308', '2729', '6125', '3227', '4557', '4104', '6691', '5511', '4137', '8048', '4138', '2916', '3356', '6577', '5508', '1102', '6776', '4163', '6538', '6732', '6782', '6523', '5236', '9942', '8112', '6189', '6585', '2211', '4766', '8081', '6592', '4105', '4968', '8436', '6446', '6170', '1522', '3213', '5880

### Trading History

In [90]:
# tidal.account.trades['3015']

### Plot chart by using Plotly

In [91]:
# tidal.analyzer.inst_chart(instrument='3015', metric_name='AdditionalInfo', plot_type=td.PlotType.LINE, scale=1.0)

### Tidal Dashboard

In [92]:
tidal.tdboard()

 * Serving Flask app 'tidal.tdboard'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:34887
 * Running on http://10.136.13.69:34887
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:10.0.10.82 - - [24/Oct/2024 13:56:15] "GET / HTTP/1.1" 200 -
INFO:werkzeug:10.0.10.82 - - [24/Oct/2024 13:56:15] "GET /static/js/main.d754b0a3.js HTTP/1.1" 200 -
INFO:werkzeug:10.0.10.82 - - [24/Oct/2024 13:56:16] "GET /static/css/main.bf4d504b.css HTTP/1.1" 200 -
INFO:werkzeug:10.0.10.82 - - [24/Oct/2024 13:56:16] "GET /images/Tidal_Logo_white.png HTTP/1.1" 200 -
INFO:werkzeug:10.0.10.82 - - [24/Oct/2024 13:56:17] "GET /api/quote/inst_list HTTP/1.1" 200 -
INFO:werkzeug:10.0.10.82 - - [24/Oct/2024 13:56:17] "GET /api/trade/trade_report HTTP/1.1" 200 -
INFO:werkzeug:10.0.10.82 - - [24/Oct/2024 13:56:17] "GET /api/metric/metric_list HTTP/1.1" 200 -
INFO:werkzeug:10.0.10.82 - - [24/Oct/2024 13:56:

In [ ]:
tidal.account.position_history